# MLP AGENT

## 1. Setup and Imports

In [1]:
import os
import sys

try:
    base_path = os.path.dirname(__file__)
except NameError:
    base_path = os.getcwd()

# 1️⃣ Ajouter le dossier parent (un cran au-dessus)
parent_dir = os.path.abspath(os.path.join(base_path, '..'))
sys.path.append(parent_dir)

# 2️⃣ Ajouter le dossier parent du dossier parent (deux crans au-dessus)
two_up_dir = os.path.abspath(os.path.join(base_path, '..', '..'))
sys.path.append(two_up_dir)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time

# Import the environment and agents
from permuted_mnist.env.permuted_mnist import PermutedMNISTEnv
from models.MLP.mlp_v0 import Agent as MLP_baseline_Agent
from models.MLP.agent_Peter_Parker import Agent as Peter_Parker_Agent
from models.MLP.agent_Bruce_Wayne import Agent as Bruce_Wayne_Agent
from models.MLP.agent_mario import Agent as Mario_Agent

print("✓ Imports successful")

✓ Imports successful


We fix the seed for reproducibility:

In [3]:
seed = 42

We fix the number of CPUs:

In [4]:
n_jobs = 2

## 2. Create the Environment

Let's create an environment with 10 different permuted tasks:

In [5]:
# Create environment with 10 episodes (tasks)
env = PermutedMNISTEnv(number_episodes=10)

# Set seed for reproducibility
env.set_seed(seed)

print(f"Environment created with {env.number_episodes} permuted tasks")
print(f"Training set size: {env.train_size} samples")
print(f"Test set size: {env.test_size} samples")

Environment created with 10 permuted tasks
Training set size: 60000 samples
Test set size: 10000 samples


## MLP :

In [6]:
def MLP(Agent):
    # Reset environment for fresh start
    env.reset()
    env.set_seed(seed)

    # Create MLP agent
    mlp_agent = Agent()

    # Track performance
    mlp_accuracies = []
    mlp_times = []

    print(f"Evaluating Agent:")

    # Evaluate on all tasks
    task_num = 1
    while True:
        task = env.get_next_task()
        if task is None:
            break
        if task_num >= 4: # limited for quick test
            break
            
        # Reset agent for new task
        mlp_agent.reset()
    
        start_time = time.time()
    
        # Train
        mlp_agent.train(task['X_train'], task['y_train'])
    
        # Make predictions
        predictions = mlp_agent.predict(task['X_test'])
    
        # Calculate time and accuracy
        elapsed_time = time.time() - start_time
        accuracy = env.evaluate(predictions, task['y_test'])
    
        mlp_accuracies.append(accuracy)
        mlp_times.append(elapsed_time)
    
        print(f"Task {task_num}: Accuracy = {accuracy:.2%}, Time = {elapsed_time:.4f}s")
        task_num += 1

    mean_accuracy = np.mean(mlp_accuracies)
    std_accuracy = np.std(mlp_accuracies)
    total_time = np.sum(mlp_times)

    print(f"\nMLP Agent Summary:")
    print(f"  Mean accuracy: {mean_accuracy:.2%} ± {std_accuracy:.2%}")
    print(f"  Total time: {total_time:.2f}s")

    return mean_accuracy

In [7]:
mean_accuracy = MLP(MLP_baseline_Agent)

Evaluating Agent:
epoch 0: 0.9664%
epoch 1: 0.9687%
epoch 2: 0.9756%
epoch 3: 0.9768%
epoch 4: 0.9775%
epoch 5: 0.9779%
epoch 6: 0.9769%
epoch 7: 0.9785%
epoch 8: 0.9794%
epoch 9: 0.9805%
Task 1: Accuracy = 98.16%, Time = 150.4125s
epoch 0: 0.9653%
epoch 1: 0.9702%
epoch 2: 0.9782%
epoch 3: 0.9786%
epoch 4: 0.9741%
epoch 5: 0.9786%
epoch 6: 0.9796%
epoch 7: 0.9818%
epoch 8: 0.9805%
epoch 9: 0.9790%
Task 2: Accuracy = 98.03%, Time = 170.2253s
epoch 0: 0.9606%
epoch 1: 0.9703%
epoch 2: 0.9765%
epoch 3: 0.9753%
epoch 4: 0.9798%
epoch 5: 0.9792%
epoch 6: 0.9802%
epoch 7: 0.9818%
epoch 8: 0.9807%
epoch 9: 0.9824%
Task 3: Accuracy = 98.23%, Time = 196.5495s

MLP Agent Summary:
  Mean accuracy: 98.14% ± 0.08%
  Total time: 517.19s


Le MLP baseline offre de bonnes performances en prédiction, mais son entraînement est trop long. Nous allons explorer des architectures plus efficaces qui réduisent le temps d’entraînement et de prédiction tout en maintenant un niveau de précision comparable.

In [8]:
mean_accuracy = MLP(Peter_Parker_Agent) 

Evaluating Agent:
Task 1: Accuracy = 96.10%, Time = 60.7351s
Task 2: Accuracy = 95.86%, Time = 59.7669s
Task 3: Accuracy = 97.44%, Time = 60.3781s

MLP Agent Summary:
  Mean accuracy: 96.47% ± 0.70%
  Total time: 180.88s


In [9]:
mean_accuracy = MLP(Bruce_Wayne_Agent) 

Evaluating Agent:
Task 1: Accuracy = 98.12%, Time = 59.6177s
Task 2: Accuracy = 98.10%, Time = 60.0172s
Task 3: Accuracy = 97.83%, Time = 59.6846s

MLP Agent Summary:
  Mean accuracy: 98.02% ± 0.13%
  Total time: 179.32s


In [10]:
mean_accuracy = MLP(Mario_Agent)

Evaluating Agent:
Task 1: Accuracy = 98.21%, Time = 60.1239s
Task 2: Accuracy = 98.07%, Time = 59.8997s
Task 3: Accuracy = 97.83%, Time = 59.5425s

MLP Agent Summary:
  Mean accuracy: 98.04% ± 0.16%
  Total time: 179.57s


Nous avons identifié des architectures de MLP plus performantes que nos modèles précédents. Ces nouvelles configurations offrent de meilleurs résultats tout en respectant les contraintes de temps, de CPU et de mémoire imposées par le problème.